In [1]:
!pip install sentence_transformers

     |████████████████████████████████| 85 kB 3.1 MB/s 
     |████████████████████████████████| 2.6 MB 39.3 MB/s 
     |████████████████████████████████| 1.2 MB 33.4 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 895 kB 28.4 MB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
     |████████████████████████████████| 636 kB 34.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126709 sha256=c0da2cd71eacc17b1be9c1fca10c98c217aeb1a6a3487330c2859743775e7c82
  Stored in directory: /root/.cache/pip/wheels/d1/c1/0f/faafd427f705c4b012274ba60d9a91d75830306811e1355293
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
filenames = ["CommentsMay2017.csv"]

In [5]:
def read_comments_from_dataset(filename):
  return pd.read_csv(filename)[["commentID", "commentBody", "commentSequence", "articleID", "parentID"]]

In [6]:
comment_sets = [(filename, read_comments_from_dataset("gdrive/MyDrive/Research/NewYorkTimesComments/" + filename)) for filename in filenames]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
len(comment_sets)

1

In [8]:
model = SentenceTransformer('stsb-mpnet-base-v2')

In [9]:
from tqdm import tqdm

def compute_embeddings(comments):
    embeddings = []

    for comment in tqdm(comments.itertuples()):
      try:
        embedding = model.encode(comment.commentBody, convert_to_tensor=True)
      except:
        embedding = model.encode("None", convert_to_tensor=True)
      embeddings.append({"ID": comment.commentID,
                      "article_id": comment.articleID,
                      "commentBody": comment.commentBody,
                      "embedding": embedding})
    return embeddings

In [10]:
for comment_set in comment_sets:
  embeddings =  []
  embedding = compute_embeddings(comment_set[1])
  embeddings.extend(embedding)
  torch.save(embeddings, "gdrive/MyDrive/Research/embedded_comments_" + comment_set[0] + "_stsb-mpnet-base-v2.pt")

276389it [1:14:23, 61.92it/s]
